In [4]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

def fetch_data(tickers, start_date="2010-01-01", end_date="2024-02-05"):
    valid_tickers = []
    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
            if data.isnull().sum().sum() == 0:
                valid_tickers.append(ticker)
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
    if not valid_tickers:
        raise ValueError("No valid tickers available for the specified date range.")
    data = yf.download(valid_tickers, start=start_date, end=end_date)['Adj Close']
    if data.isnull().sum().sum() > 0:
        print("Warning: NaN values found and are being dropped.")
        data = data.dropna()
    return data

def simulate_portfolios(daily_returns, num_portfolios=5000, num_days=252*5):
    mean_returns = daily_returns.mean()
    cov_matrix = daily_returns.cov()
    
    portfolio_returns = []
    portfolio_volatilities = []
    sharpe_ratios = []
    stock_weights = []

    for _ in range(num_portfolios):
        weights = np.random.random(len(daily_returns.columns))
        weights /= np.sum(weights)
        simulated_returns = np.random.multivariate_normal(mean_returns, cov_matrix, num_days)
        cumulative_returns = (simulated_returns @ weights + 1).cumprod() - 1
        
        port_return = np.sum(weights * mean_returns) * 252
        port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        
        sharpe_ratio = port_return / port_volatility
        
        portfolio_returns.append(port_return)
        portfolio_volatilities.append(port_volatility)
        sharpe_ratios.append(sharpe_ratio)
        stock_weights.append(weights)

    portfolio_data = {
        'Returns': portfolio_returns, 
        'Volatility': portfolio_volatilities, 
        'Sharpe Ratio': sharpe_ratios
    }
    for counter, symbol in enumerate(daily_returns.columns):
        portfolio_data[symbol] = [w[counter] for w in stock_weights]
        
    df = pd.DataFrame(portfolio_data)
    
    return df

def plot_results(df, tickers):
    with PdfPages("C:\\Users\\demar\\Downloads\\portfolio_optimization_report.pdf") as pdf:
        # Efficient Frontier with Highlighted Optimal and GMVP Portfolios
        plt.figure(figsize=(12, 8))
        sns.scatterplot(x='Volatility', y='Returns', hue='Sharpe Ratio', data=df, palette='viridis', edgecolor=None, alpha=0.7)
        max_sharpe = df['Sharpe Ratio'].idxmax()
        plt.scatter(df.loc[max_sharpe, 'Volatility'], df.loc[max_sharpe, 'Returns'], color='red', s=100, marker='*', label=f'Max Sharpe Ratio Portfolio (Sharpe Ratio: {df.loc[max_sharpe, "Sharpe Ratio"]:.2f})')
        min_volatility = df['Volatility'].idxmin()
        plt.scatter(df.loc[min_volatility, 'Volatility'], df.loc[min_volatility, 'Returns'], color='blue', s=100, marker='o', label=f'Global Minimum Variance Portfolio (Volatility: {df.loc[min_volatility, "Volatility"]:.2f})')
        plt.title("Efficient Frontier with Highlighted Optimal and GMVP Portfolios")
        plt.xlabel("Annualized Volatility")
        plt.ylabel("Annualized Returns")
        plt.legend()
        plt.tight_layout()
        pdf.savefig()
        plt.close()

        # Efficient Frontier Analysis
        plt.figure(figsize=(12, 6))
        plt.axis('off')
        plt.text(0.05, 0.95, 
            "Analysis: The efficient frontier, as visualized in this scatter plot, is foundational in modern portfolio theory. It represents a boundary on a risk-return plane where no other portfolios exist with a higher expected return for the same level of risk. Each point on the frontier offers the highest possible expected return for its level of risk. The red star signifies the portfolio with the maximum Sharpe ratio—a key metric in finance that quantifies the risk-adjusted return of an investment. The blue circle, on the other hand, marks the Global Minimum Variance Portfolio (GMVP). This is the point on the frontier with the absolute lowest risk.", 
            wrap=True, horizontalalignment='left', fontsize=10, verticalalignment='top')
        pdf.savefig()
        plt.close()

        # Stock Allocation for Maximum Sharpe Ratio Portfolio
        allocation_max_sharpe = df.loc[max_sharpe, tickers].sort_values(ascending=False)
        plt.figure(figsize=(12,6))
        sns.barplot(x=allocation_max_sharpe.index, y=allocation_max_sharpe.values, color="blue")
        plt.title("Stock Allocation for Maximum Sharpe Ratio Portfolio")
        plt.ylabel("Weight")
        plt.xlabel("Stock")
        for i, value in enumerate(allocation_max_sharpe.values):
            plt.text(i, value + 0.01, f"{value*100:.2f}%", ha='center', va='bottom', fontsize=12)
        plt.tight_layout()
        pdf.savefig()
        plt.close()

        # Max Sharpe Ratio Portfolio Analysis
        plt.figure(figsize=(12, 6))
        plt.axis('off')
        plt.text(0.05, 0.95, 
            "Analysis: This bar chart delves into the portfolio composition of the one with the maximum Sharpe ratio, shedding light on the specific weightings of individual stocks. Stock allocation in a portfolio is paramount, as it directly influences the portfolio's overall risk and return characteristics. The height of each bar signifies the percentage of the portfolio's total value that's allocated to a particular stock. A higher allocation to a specific stock suggests that, given the historical data and our assumptions, this stock contributes more significantly to enhancing the portfolio's risk-adjusted return.", 
            wrap=True, horizontalalignment='left', fontsize=10, verticalalignment='top')
        pdf.savefig()
        plt.close()

        # Stock Allocation for GMVP
        allocation_gmvp = df.loc[min_volatility, tickers].sort_values(ascending=False)
        plt.figure(figsize=(12,6))
        sns.barplot(x=allocation_gmvp.index, y=allocation_gmvp.values, color="red")
        plt.title("Stock Allocation for Global Minimum Variance Portfolio")
        plt.ylabel("Weight")
        plt.xlabel("Stock")
        for i, value in enumerate(allocation_gmvp.values):
            plt.text(i, value + 0.01, f"{value*100:.2f}%", ha='center', va='bottom', fontsize=12)
        plt.tight_layout()
        pdf.savefig()
        plt.close()

        # GMVP Analysis
        plt.figure(figsize=(12, 6))
        plt.axis('off')
        plt.text(0.05, 0.95, 
            "Analysis: This visualization breaks down the composition of the Global Minimum Variance Portfolio (GMVP), emphasizing the diversification strategy that minimizes the total portfolio risk. The GMVP is a fascinating construct in portfolio management, as it solely focuses on minimizing risk, regardless of the returns. Each bar in the chart represents the proportion of the portfolio invested in a specific stock. Diversification is a core principle here: by spreading investments across various assets, the GMVP aims to mitigate unsystematic risks associated with individual stocks.", 
            wrap=True, horizontalalignment='left', fontsize=10, verticalalignment='top')
        pdf.savefig()
        plt.close()

def main():
    sns.set_style("whitegrid")
    stocks = ["ATRFX", "DBC", "COPX", "CPER", "AVDV", "DBMF", "AVEM", "KWEB", "TMV", "PMF", "TLT", "KRBN", "SOXS", "GOOG", "NESN.SW", "ROG.SW", "BTC-USD", "VXX"]
    data = fetch_data(stocks)
    daily_returns = data.pct_change().dropna()
    df = simulate_portfolios(daily_returns)
    plot_results(df, daily_returns.columns)

if __name__ == "__main__":
    main()




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******